In [14]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K
from keras.datasets import mnist
import numpy as np
from keras.callbacks import TensorBoard

In [47]:
labels=['violent','non_violent']

X_train = np.load('/media/raihan/work/Projects/499/violence-detection/30x30_10FPS/train/'+labels[0] + '.npy')
y_train = np.zeros(X_train.shape[0])

for i, label in enumerate(labels[1:]):
    x = np.load('/media/raihan/work/Projects/499/violence-detection/30x30_10FPS/train/'+label + '.npy')
    X_train = np.vstack((X_train, x))
    y_train = np.append(y_train, np.full(x.shape[0], fill_value= (i + 1)))

In [50]:
X_test = np.load('/media/raihan/work/Projects/499/violence-detection/30x30_10FPS/test/'+labels[0] + '.npy')
y_test = np.zeros(X_test.shape[0])

for i, label in enumerate(labels[1:]):
    x = np.load('/media/raihan/work/Projects/499/violence-detection/30x30_10FPS/test/'+label + '.npy')
    X_test = np.vstack((X_test, x))
    y_test = np.append(y_test, np.full(x.shape[0], fill_value= (i + 1)))

In [59]:
x_train = X_train.astype('float32') / 255.
x_test = X_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 30, 30, 1))
x_test = np.reshape(x_test, (len(x_test), 30, 30, 1))

In [82]:
input_img = Input(shape=(30,30,1))

x = Conv2D(32, (2,2), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2,2), padding='same')(x)
x = Conv2D(16, (2,2), activation='relu', padding='same')(x)
x = MaxPooling2D((2,2), padding='same')(x)
x = Conv2D(8, (2,2), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2,2), padding='same')(x)

x = Conv2D(8, (2,2), activation='relu',padding='same')(encoded)
x = UpSampling2D((2,2))(x)
x = Conv2D(16, (2,2), activation='relu', padding='same')(x)
x = UpSampling2D((2,2))(x)
x = Conv2D(32, (2,2), activation='relu')(x)
x = UpSampling2D((2,2))(x)
decoded = Conv2D(1, (2,2), activation='sigmoid', padding='same')(x)

In [90]:
autoencoder = Model(input_img, decoded)
autoencoder.summary()
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_21 (InputLayer)        (None, 30, 30, 1)         0         
_________________________________________________________________
conv2d_134 (Conv2D)          (None, 30, 30, 32)        160       
_________________________________________________________________
max_pooling2d_61 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_135 (Conv2D)          (None, 15, 15, 16)        2064      
_________________________________________________________________
max_pooling2d_62 (MaxPooling (None, 8, 8, 16)          0         
_________________________________________________________________
conv2d_136 (Conv2D)          (None, 8, 8, 8)           520       
_________________________________________________________________
max_pooling2d_63 (MaxPooling (None, 4, 4, 8)           0         
__________

In [97]:
autoencoder.fit(x_train,x_train,
                epochs = 50,
                batch_size = 128,
                shuffle=True,
                validation_data = (x_test,x_test),
                callbacks=[TensorBoard(log_dir='/tmp/autoencoder')],
                verbose=1)

Train on 26745 samples, validate on 3924 samples
Epoch 1/50
26745/26745 [==============================] - 38s 1ms/step - loss: 0.5428 - acc: 0.0960 - val_loss: 0.6019 - val_acc: 0.0388
Epoch 2/50
26745/26745 [==============================] - 42s 2ms/step - loss: 0.5423 - acc: 0.0961 - val_loss: 0.6001 - val_acc: 0.0386
Epoch 3/50
26745/26745 [==============================] - 42s 2ms/step - loss: 0.5421 - acc: 0.0961 - val_loss: 0.5994 - val_acc: 0.0388
Epoch 4/50
26745/26745 [==============================] - 41s 2ms/step - loss: 0.5418 - acc: 0.0961 - val_loss: 0.6011 - val_acc: 0.0386
Epoch 5/50
26745/26745 [==============================] - 40s 2ms/step - loss: 0.5414 - acc: 0.0961 - val_loss: 0.5989 - val_acc: 0.0388
Epoch 6/50
26745/26745 [==============================] - 41s 2ms/step - loss: 0.5408 - acc: 0.0962 - val_loss: 0.5997 - val_acc: 0.0387
Epoch 7/50
26745/26745 [==============================] - 40s 1ms/step - loss: 0.5407 - acc: 0.0962 - val_loss: 0.5988 - val_acc: